### xxxxx

In [1]:
# Benötigte Pakete importieren
# "Standard"-import
import os
import pandas as pd
# Einlesen & entpacken in html-Dateien
from glob import glob
import tarfile
# Kreation Mini-Data-Warehouse (SQL-Datenbank)
from datetime import datetime
import sqlite3
# Bearbeiten von html-Dateien
from bs4 import BeautifulSoup
import requests

In [2]:
# Pfade für das Projekt 

# Pfad für das Projekt
PROJECT_ROOT = os.getcwd()  

# Input, Rohdaten & html-Dateien
INPUT_PATH = os.path.join(PROJECT_ROOT, "input", "raw")
ZIP_PATH = os.path.join(INPUT_PATH, "downloaded_zips")
DATA_LAKE_PATH = os.path.join(INPUT_PATH, "data-lake")  

# Mini-Data-Warehouse bzw. SQL-Datenbank
LOGFILE_PATH = os.path.join(DATA_LAKE_PATH, "2025-02-08.csv")
SQL_PATH = os.path.join(PROJECT_ROOT, "output", "dwh.sqlite3")



In [3]:
# Extrahieren & Einlesen einer .tar.gz-Datei
def extract_tar_file(ZIP_PATH, DATA_LAKE_PATH): 
    with tarfile.open(ZIP_PATH, "r:gz") as tar:
        for member in tar.getmembers():
            # Zielpfad für jede Datei bestimmen
            filename = os.path.basename(member.name)
            
            # Nur HTML-Dateien entpacken
            if filename.endswith(".html"):
                target_path = os.path.join(DATA_LAKE_PATH, filename)
                
                # Skippen, wenn Datei schon existiert
                if os.path.exists(target_path):
                    continue
                
                # Entpacken in HTML-Datei
                with open(target_path, "wb") as f:
                    f.write(tar.extractfile(member).read())
                
                print(f"Entpackt: {filename}")

In [4]:
# html-Datei mit Encoding einlesen
def read_html_file(filepath, encoding="utf-8"):
    # Datei öffnen und einlesen
    with open(filepath, "r", encoding=encoding) as f:
        return f.read()

# html-Inhalt bereinigen & in einzelne Wörter zerlegen
def process_html(html):
    # html-Parser von BeautifulSoup nutzen, um nur den Text auszulesen
    bstext = BeautifulSoup(html, "html.parser")
    text = bstext.get_text(separator=" ").lower()

    # Zeilenumbrüche durch Leerzeichen ersetzen & Wörter trennen
    items = text.replace("\n", " ").split(" ")

    # Stoppwörter importieren
    stopwords_url = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
    now_str = "2025-02-08"
    stopwords_list = requests.get(stopwords_url, allow_redirects=True).text.split("\n")[9:]

    # Kurze Wörter und Stoppwörter entfernen
    items = [i for i in items if len(i) > 1 and i not in stopwords_list]
    return items

In [5]:
one_row = log_file.iloc[0]  # oder log_file[log_file["name"] == "sz"].iloc[0]
print(one_row)

NameError: name 'log_file' is not defined

In [ ]:
test_path = os.path.join(DATA_LAKE_PATH, os.path.basename(one_row["file_name"]))
test_encoding = one_row["encoding"]
print(test_path)
print(test_encoding)

c:\Users\Marta\Documents\Marta\DBU\ADSC11 ADS-01\Studienarbeit\newspaper-scraping\input\raw\data-lake\2025-02-08-sz.html
utf-8


In [ ]:
html = read_html_file(test_path, test_encoding)
print(html[:500])  # Nur die ersten 500 Zeichen anzeigen

<!DOCTYPE html><html lang="de"><head><script type="text/javascript">window.dataLayer = [{"page.publisher":"sz","page.platform_type":"standard","page.brand_product":"sz_de","page.object_id":"8","page.docType":"homepage","page.tech":"homie","tech.robots_status":"true","ads.adtag":"true"}]</script><meta data-meta-viewport="true" name="viewport" content="width=device-width, initial-scale=1"/>
  <!--
        ____               _      __           ___      __       __         _        __             _


In [ ]:
# Einzelne Zeitungsdateien verarbeiten 
def process_newspaper(newspaper):
    filename = os.path.basename(newspaper["file_name"])  # Nur den Dateinamen rausholen
    full_path = os.path.join(DATA_LAKE_PATH, filename)   

    # Encoding aus der CSV einlesen
    encoding = newspaper["encoding"].lower()
    
    # html-Datei laden & bereinigen
    html = read_html_file(full_path, encoding)
    items = process_html(html)

   # Häufigkeit jedes Wortes zählen
    count = pd.Series(items).value_counts()

    # Als DataFrame formatieren und Zusatzinfos ergänzen
    count_df = count.to_frame()
    count_df.columns = ["count"]
    count_df["word"] = count_df.index
    count_df["paper"] = newspaper["name"]
    count_df["date"] = newspaper["date"]

    return count_df

In [ ]:
one_row = log_file.iloc[0]
print(one_row)

Unnamed: 0                                 0
name                                      sz
date                              2025-02-08
file_name       data-lake/2025-02-08-sz.html
status                                   200
original_url    https://www.sueddeutsche.de/
final_url       https://www.sueddeutsche.de/
encoding                               utf-8
Name: 0, dtype: object


In [ ]:
df_test = process_newspaper(one_row)
df_test.head()

,count,word,paper,date
sz,142,sz,sz,2025-02-08
plus,121,plus,sz,2025-02-08
anzeigen,26,anzeigen,sz,2025-02-08
magazin,10,magazin,sz,2025-02-08
artikel,9,artikel,sz,2025-02-08


In [ ]:
# Liste für die Ergebnisse-Sammlung erstellen
collection = []

In [ ]:
# Verarbeitung jeder einzelnen Datei und DataFrame mit Wortzählungen zurückgeben
def process_wrapper(newspaper):
    name = newspaper["name"]
    try:
        count = process_newspaper(newspaper)
        print(f"[INFO] Verarbeitung erfolgreich: {name}")
        collection.append(count)
    except Exception as e:
        print(f"[ERROR] Fehler bei {name}: {e}")

In [ ]:
# Alle CSV-Dateien im data-lake automatisch durchgehen
csv_files = sorted([f for f in os.listdir(DATA_LAKE_PATH) if f.endswith(".csv")])

for file in csv_files:
    path = os.path.join(DATA_LAKE_PATH, file)
    try:
        log_file = pd.read_csv(path)
        log_file.apply(process_wrapper, axis=1)
        print(f"[INFO] verarbeitet: {file}")
    except Exception as e:
        print(f"[ERROR] Fehler bei Datei {file}: {e}")

[INFO] Verarbeitung erfolgreich: sz
[INFO] Verarbeitung erfolgreich: zeit
[INFO] Verarbeitung erfolgreich: faz
[INFO] Verarbeitung erfolgreich: heise
[INFO] Verarbeitung erfolgreich: golem
[INFO] Verarbeitung erfolgreich: tagesspiegel
[INFO] Verarbeitung erfolgreich: taz
[INFO] Verarbeitung erfolgreich: abendblatt
[INFO] Verarbeitung erfolgreich: berliner
[INFO] Verarbeitung erfolgreich: welt
[INFO] Verarbeitung erfolgreich: medium
[INFO] Verarbeitung erfolgreich: esslinger
[INFO] Verarbeitung erfolgreich: kdnuggets
[INFO] Verarbeitung erfolgreich: handelsblatt
[INFO] Verarbeitung erfolgreich: ntv
[INFO] Verarbeitung erfolgreich: pioneer
[INFO] Verarbeitung erfolgreich: suedwest
[INFO] Verarbeitung erfolgreich: t3n
[INFO] Verarbeitung erfolgreich: economist
[INFO] Verarbeitung erfolgreich: srf
[INFO] Verarbeitung erfolgreich: wef
[INFO] Verarbeitung erfolgreich: stuttgarter
[INFO] Verarbeitung erfolgreich: atlantic
[INFO] Verarbeitung erfolgreich: netzpolitik
[INFO] Verarbeitung erfolg

In [6]:
csv_files = sorted([f for f in os.listdir(DATA_LAKE_PATH) if f.endswith(".csv")])

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'c:\\Users\\Marta\\Documents\\Marta\\DBU\\ADSC11 ADS-01\\Studienarbeit\\newspaper-scraping\\scripts\\input\\raw\\data-lake'

In [ ]:
# Alle Ergebnisse in einer Tabelle sammeln
if collection:
    data = pd.concat(collection, axis=0)
    print("Data shape:", data.shape)
    print("Zeitraum:", data["date"].min(), "bis", data["date"].max())
else:
    print("Keine Daten verarbeitet – Sammlung ist leer.")

In [ ]:
# In Datenbank speichern
# Erstellung SQL-Datenbank (dwh.sqlite3)
connection = sqlite3.connect(SQL_PATH)

# Wordcount-DataFrame in eine Tabelle erstellen
data.to_sql("wordcount", connection, index=False, if_exists="append")

53939

In [ ]:
print("SQL-Pfad:", SQL_PATH)
print("Existiert die Datei?", os.path.exists(SQL_PATH))

SQL-Pfad: c:\Users\Marta\Documents\Marta\DBU\ADSC11 ADS-01\Studienarbeit\newspaper-scraping\output\dwh.sqlite3
Existiert die Datei? True


In [ ]:
conn = sqlite3.connect(SQL_PATH)
cursor = conn.cursor()

# Tabellen im DWH anzeigen
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tabellen:", cursor.fetchall())

Tabellen: [('wordcount',)]


In [ ]:
# Zeige 5 Zeilen aus der Tabelle 'wordcount'
cursor.execute("SELECT * FROM wordcount LIMIT 5;")
rows = cursor.fetchall()

for row in rows:
    print(row)

(142, 'sz', 'sz', '2025-02-08')
(121, 'plus', 'sz', '2025-02-08')
(26, 'anzeigen', 'sz', '2025-02-08')
(10, 'magazin', 'sz', '2025-02-08')
(9, 'artikel', 'sz', '2025-02-08')
